In [10]:
%env SPARK_HOME=/usr/hdp/current/spark2-client

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, column, max, min

spark = SparkSession.builder.getOrCreate()
print(spark)

env: SPARK_HOME=/usr/hdp/current/spark2-client


In [11]:
users_raw = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Users.csv")

users_raw.printSchema()

users = users_raw.select(\
    users_raw['User-ID'].cast('integer').alias('userId'),\
    users_raw['Age'].cast('integer').alias('age'),\
)
users.printSchema()
users.show(5)

root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- age: integer (nullable = true)

+------+----+
|userId| age|
+------+----+
|     1|null|
|     2|  18|
|     3|null|
|     4|  17|
|     5|null|
+------+----+
only showing top 5 rows



In [30]:
ratings_raw = spark.read.format("csv")\
    .option("sep",";")\
    .option("header", "true")\
    .load("/data/books/BX-Book-Ratings.csv")

ratings_raw.printSchema()
print(ratings_raw.count())

ratings = ratings_raw.select(\
    ratings_raw['User-ID'].cast('integer').alias('userId'),\
    ratings_raw['ISBN'].cast('integer').alias('itemId'),\
    ratings_raw['Book-Rating'].cast('integer').alias('rating'),\
)
ratings.printSchema()


root
 |-- User-ID: string (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: string (nullable = true)

1149780
root
 |-- userId: integer (nullable = true)
 |-- itemId: integer (nullable = true)
 |-- rating: integer (nullable = true)



In [31]:
ratings_filtered = ratings_raw.rdd\
    .filter(lambda row: (row['userId'] is not None))\
    .filter(lambda row: (row['itemId'] is not None))\
    .filter(lambda row: (row['rating'] is not None))\
    .toDF()

print(ratings_filtered.count())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 274.0 failed 4 times, most recent failure: Lost task 0.3 in stage 274.0 (TID 1722, c7203, executor 2): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/hadoop/yarn/local/usercache/vagrant/appcache/application_1571692004819_0001/container_1571692004819_0001_01_000003/pyspark.zip/pyspark/sql/types.py", line 1543, in __getitem__
    idx = self.__fields__.index(item)
ValueError: 'userId' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/hadoop/yarn/local/usercache/vagrant/appcache/application_1571692004819_0001/container_1571692004819_0001_01_000003/pyspark.zip/pyspark/worker.py", line 253, in main
    process()
  File "/hadoop/yarn/local/usercache/vagrant/appcache/application_1571692004819_0001/

In [28]:
ratings_grouped = ratings_filtered\
    .groupBy("userId")\
    .count()\
    .rdd\
    .filter(lambda row: (row['count'] > 1))\
    .toDF()
print(ratings_grouped.count())


38462


In [17]:
from pyspark.ml.recommendation import ALS

training, test = ratings_filtered.randomSplit([0.8, 0.2])
als = ALS()\
    .setMaxIter(5)\
    .setRegParam(0.01)\
    .setUserCol("userId")\
    .setItemCol("itemId")\
    .setRatingCol("rating")

alsModel = als.fit(training)
predictions = alsModel.transform(test)
print(predictions)

DataFrame[userId: bigint, itemId: bigint, rating: bigint, prediction: float]


In [18]:
predictions.show(5)

+------+-------+------+----------+
|userId| itemId|rating|prediction|
+------+-------+------+----------+
| 66619| 225755|     0|       NaN|
|190079|2232545|     0|       NaN|
|189334|2250810|     0| 3.0840244|
| 53729|2323338|     0|       NaN|
| 63404|2740958|     0|       NaN|
+------+-------+------+----------+
only showing top 5 rows



In [19]:
print(predictions)

DataFrame[userId: bigint, itemId: bigint, rating: bigint, prediction: float]


In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()\
    .setMetricName("rmse")\
    .setLabelCol("rating")\
    .setPredictionCol("prediction")

rmse = evaluator.evaluate(predictions)
print("RMSE = %f" % rmse)

RMSE = nan
